In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
pwd

'/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Advanced Business Analytics/Project/Business_Analytics_Olist/phase2'

In [118]:
df_reviews = pd.read_csv("/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Advanced Business Analytics/Project/data/olist_order_reviews_dataset.csv" ,sep=',')
df_orders = pd.read_csv("/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Advanced Business Analytics/Project/data/olist_orders_dataset.csv" ,sep=',')
df_items = pd.read_csv("/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Advanced Business Analytics/Project/data/olist_order_items_dataset.csv" ,sep=',')
df_products = pd.read_csv("/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Advanced Business Analytics/Project/data/olist_products_dataset.csv" ,sep=',')
#df_customer = pd.read_csv("/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Advanced Business Analytics/Project/data/olist_customers_dataset.csv" ,sep=',')

In [11]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [10]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [8]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [9]:
df_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [13]:
#df_customer.head()

In [103]:
df_items.describe(include="O")

,order_id,product_id,seller_id,shipping_limit_date
count,112650,112650,112650,112650
unique,98666,32951,3095,93318
top,8272b63d03f5f79c56e9e4120aec44ef,aca2eb7d00ea1a7b8ebd4e68314663af,6560211a19b47992c3666cc44a7e94c0,2017-07-21 18:25:23
freq,21,527,2033,21


In [102]:
df_orders.describe(include="O")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,acbd43df296646b7b242d1e25b18a646,de5af6c03310241012f6c59998a13089,delivered,2018-08-02 12:06:07,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-02-14 21:09:19,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [104]:
df_reviews.describe(include="O")

,review_id,order_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,100000,100000,11715,41753,100000,100000
unique,99173,99441,4600,36921,637,99010
top,4d0e6dd087008d1f992d25ef6e1f619f,8e17072ec97ce29f0e1f111e598b0c85,Recomendo,Muito bom,2017-12-19 00:00:00,2017-06-15 23:21:05
freq,3,3,426,230,466,4


In [109]:
df_reviews[df_reviews['order_id']=='8e17072ec97ce29f0e1f111e598b0c85']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
45043,67c2557eb0bd72e3ece1e03477c9dff5,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,Entregou o produto errado.,2018-04-07 00:00:00,2018-04-08 22:48:27
65024,2d6ac45f859465b5c185274a1c929637,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,Comprei 3 unidades do produto vieram 2 unidade...,2018-04-07 00:00:00,2018-04-07 21:13:05
93027,6e4c4086d9611ae4cc0cc65a262751fe,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,"Embora tenha entregue dentro do prazo, não env...",2018-04-14 00:00:00,2018-04-16 11:37:31


In [124]:
df_reviews[df_reviews['review_id']=='4d0e6dd087008d1f992d25ef6e1f619f']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
7708,4d0e6dd087008d1f992d25ef6e1f619f,d415e95816c2f3a2a34e06cd519157cc,4,NaN,NaN,2018-06-12 00:00:00,2018-06-17 23:48:04
15583,4d0e6dd087008d1f992d25ef6e1f619f,cd0343141ac675cd40fab2c7fd2e99cc,4,NaN,NaN,2018-06-12 00:00:00,2018-06-17 23:48:04
50598,4d0e6dd087008d1f992d25ef6e1f619f,97f95535ada31d6fa853230e815ee4e3,4,NaN,NaN,2018-06-12 00:00:00,2018-06-17 23:48:04


In [110]:
df_orders[df_orders['order_id']=='8e17072ec97ce29f0e1f111e598b0c85']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
63380,8e17072ec97ce29f0e1f111e598b0c85,9e29cde4ddb42f9330bcf3c9bc23d177,delivered,2018-03-31 15:08:21,2018-03-31 15:50:11,2018-04-03 15:58:24,2018-04-06 17:38:34,2018-05-04 00:00:00


In [108]:
len(df_reviews.drop_duplicates())

100000

In [48]:
len(df_orders)

99441

In [50]:
len(df_reviews)

100000

In [54]:
len(df_items)

112650

In [55]:
len(df_products)

32951

In [56]:
df_reviews.order_id.nunique()

99441

In [78]:
scores = pd.DataFrame(df_reviews.groupby('order_id')['review_score'].count().sort_values())

In [79]:
scores.head()

,review_score
order_id,
00010242fe8c5a6d1ba2dd792cb16214,1
ab010289ce552581dd007c7276d4e276,1
ab001f825765f185062e85eaebc010b5,1
aaff8afa47c8426e414a6d908a97713c,1
aaff827d79b2aca52ee986179a052077,1


In [84]:
more_scores = scores[scores.review_score>1]

In [92]:
order_numbers = list(more_scores.index)

In [112]:
df_reviews[df_reviews.order_id == order_numbers[-1]]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1997,ffb8cff872a625632ac983eb1f88843c,c88b1d1b157a9999ce368f218a407141,3,NaN,NaN,2017-07-22 00:00:00,2017-07-26 13:41:07
83187,202b5f44d09cd3cfc0d6bd12f01b044c,c88b1d1b157a9999ce368f218a407141,5,NaN,NaN,2017-07-22 00:00:00,2017-07-26 13:40:22
90069,fb96ea2ef8cce1c888f4d45c8e22b793,c88b1d1b157a9999ce368f218a407141,5,NaN,NaN,2017-07-21 00:00:00,2017-07-26 13:45:15


In [127]:
df[df['review_id']=='4d0e6dd087008d1f992d25ef6e1f619f']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
65856,97f95535ada31d6fa853230e815ee4e3,a0d04a491f25030236e94355ba1e7e47,delivered,2018-06-01 13:39:44,2018-06-01 13:55:09,2018-06-04 14:29:00,2018-06-11 21:33:15,2018-07-12 00:00:00,4d0e6dd087008d1f992d25ef6e1f619f,4,NaN,NaN,2018-06-12 00:00:00,2018-06-17 23:48:04
82115,cd0343141ac675cd40fab2c7fd2e99cc,0a84514b054a38a558ad0874beebf6af,delivered,2018-06-01 13:39:44,2018-06-01 13:52:38,2018-06-04 14:29:00,2018-06-11 21:28:48,2018-07-12 00:00:00,4d0e6dd087008d1f992d25ef6e1f619f,4,NaN,NaN,2018-06-12 00:00:00,2018-06-17 23:48:04
82915,d415e95816c2f3a2a34e06cd519157cc,f0ab4acc7e4a8236d84d6f74bad11b0e,delivered,2018-06-01 13:39:44,2018-06-01 13:52:44,2018-06-04 14:29:00,2018-06-11 19:53:41,2018-07-12 00:00:00,4d0e6dd087008d1f992d25ef6e1f619f,4,NaN,NaN,2018-06-12 00:00:00,2018-06-17 23:48:04


In [101]:
df_orders[df_orders.order_id=="c88b1d1b157a9999ce368f218a407141"]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
37179,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,delivered,2017-07-17 22:44:23,2017-07-18 22:50:12,2017-07-20 15:27:41,2017-07-21 17:06:30,2017-08-01 00:00:00


In [123]:
#type(df_reviews.order_id[0])

In [125]:
#Merging reviews on orders
df = df_orders.merge(df_reviews, on="order_id", how= "inner")
len(df)

100000

In [126]:
df[df.order_id=="c88b1d1b157a9999ce368f218a407141"]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
37370,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,delivered,2017-07-17 22:44:23,2017-07-18 22:50:12,2017-07-20 15:27:41,2017-07-21 17:06:30,2017-08-01 00:00:00,ffb8cff872a625632ac983eb1f88843c,3,NaN,NaN,2017-07-22 00:00:00,2017-07-26 13:41:07
37371,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,delivered,2017-07-17 22:44:23,2017-07-18 22:50:12,2017-07-20 15:27:41,2017-07-21 17:06:30,2017-08-01 00:00:00,202b5f44d09cd3cfc0d6bd12f01b044c,5,NaN,NaN,2017-07-22 00:00:00,2017-07-26 13:40:22
37372,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,delivered,2017-07-17 22:44:23,2017-07-18 22:50:12,2017-07-20 15:27:41,2017-07-21 17:06:30,2017-08-01 00:00:00,fb96ea2ef8cce1c888f4d45c8e22b793,5,NaN,NaN,2017-07-21 00:00:00,2017-07-26 13:45:15


In [128]:
print("products: ", len(df_products), "items: ", len(df_items))

products:  32951 items:  112650


In [35]:
#Merging product categories on items
items = df_products.merge(df_items, on= "product_id", how="left")

In [130]:
items.describe(include="O")

,product_id,product_category_name,order_id,seller_id,shipping_limit_date
count,112650,111047,112650,112650,112650
unique,32951,73,98666,3095,93318
top,aca2eb7d00ea1a7b8ebd4e68314663af,cama_mesa_banho,8272b63d03f5f79c56e9e4120aec44ef,6560211a19b47992c3666cc44a7e94c0,2018-03-01 02:50:48
freq,527,11115,21,2033,21


In [36]:
items.columns[2:-6]

Index(['product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],
      dtype='object')

In [37]:
#Dropping columns that does not matter
cols = items.columns[2:-6]
items = items.drop(cols,axis=1)

In [38]:
#Merging the items onto the orders:
df = df.merge(items, on = "order_id", how="left")

In [39]:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,...,review_comment_message,review_creation_date,review_answer_timestamp,product_id,product_category_name,order_item_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,a54f0611adc9ed256b57ede6b6eb5114,4,...,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,87285b34884572647811a353c7ac498a,utilidades_domesticas,1.0,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,8d5266042046a06655c8db133d120ba5,4,...,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,595fac2a385ac33a80bd5114aec74eb8,perfumaria,1.0,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,e73b67b67587f7644d5bd1a52deb1b01,5,...,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,aa4383b373c6aca5d8797843e5594415,automotivo,1.0,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,359d03e676b3c069f62cadba8dd3f6e8,5,...,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop,1.0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,e50934924e227544ba8246aeb3770dd4,5,...,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51,65266b2da20d04dbe00c5c2d3bb7859e,papelaria,1.0,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [40]:
df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'product_id', 'product_category_name',
       'order_item_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value'],
      dtype='object')

In [42]:
dfs = df[["order_id", "customer_id", "review_score", "product_id", "product_category_name", "order_item_id"]]

In [43]:
dfs.head()

,order_id,customer_id,review_score,product_id,product_category_name,order_item_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,4,87285b34884572647811a353c7ac498a,utilidades_domesticas,1.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,4,595fac2a385ac33a80bd5114aec74eb8,perfumaria,1.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,5,aa4383b373c6aca5d8797843e5594415,automotivo,1.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,5,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop,1.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,5,65266b2da20d04dbe00c5c2d3bb7859e,papelaria,1.0


In [114]:
dfs.describe(include="O")

,order_id,customer_id,product_id,product_category_name
count,114100,114100,113322,111710
unique,99441,99441,32951,73
top,5a3b1c29a49756e75f1ef513383c0c12,be1c4e52bb71e0c54b11a26b8e8d59f2,aca2eb7d00ea1a7b8ebd4e68314663af,cama_mesa_banho
freq,22,22,527,11272


In [44]:
len(dfs)

114100

### Collaborative filtering 

Collaborative recommenders rely on data generated by users as they interact with items. 

Benefits 

- it is always “self-generating” — users create the data for you naturally as they interact with items. This can be a valuable data source, especially in cases where high-quality item features are not available or difficult to obtain. 
- Another benefit of collaborative filters is that it helps users discover new items that are outside the subspace defined by their historical profile.

Drawbacks 

- the well-known cold start problem. 
- It is also difficult for collaborative filters to accurately recommend novel or niche items because these items typically do not have enough user-item interaction data.

**Item-item** (http://www.diva-portal.org/smash/get/diva2:1111865/FULLTEXT01.pdf):

Item based collaborative filtering was introduced 1998 by Amazon[6]. Unlike user based collaborative filtering, item based filtering looks at the similarity between different items, and does this by taking note of how many users that bought item X also bought item Y. If the correlation is high enough, a similarity can be presumed to exist between the two items, and they can be assumed to be similar to one another. Item Y will from there on be recommended to users who bought item X and vice versa.

In item-item collaborative filtering, we provide a recommendation based on other items similar to ours. The **benefits** of it, compared to user-user collaborative filtering, is that we usually need much fewer similarity computations (in most cases, there are much more users in systems than items). The most **common pitfall** - the system can provide very obvious recommendations.

**User-user** (http://www.diva-portal.org/smash/get/diva2:1111865/FULLTEXT01.pdf):

The report is focusing on the “nearest neighbour” approach for recommendations, which looks at the users rating patterns and finds the “nearest neighbours”, i.e users with ratings similar to yours. The algorithm then proceeds to give you recommendations based on the ratings of these neighbours.

In user-user collaborative filtering, we provide a recommendation based on tastes of other users similar to us. **The problem** with that algorithm is that we need a lot of information about other people to provide correct recommendations, but the main benefits are effectiveness and ability to provide new, unexpected, and, yet, good recommendations.

### Content recommenders

Content recommenders rely on item features to make recommendations. 

Benefits

- Content filters tend to be more robust against popularity bias and the cold start problem. 
- They can easily recommend new or novel items based on niche tastes. 

Drawbacks

- However, in an item-to-item recommender, content filters can only recommend items with features similar to the original item. 
- This limits the scope of recommendations, and can also result in surfacing items with low ratings.

##### In this project

Given our dataset and the features we have, we are not able to create a content-based filtering algorithm, as the algorithm would need more information. We would essentially need to know some specific attributes about every product, e.g. with movies we know whether they include themes such as: Baseball, Economics, etc. 
Furthermore, companies such as Amazon are using collaborative item-item based recommender systems (https://www.quora.com/What-algorithm-s-does-Amazon-use-in-their-recommendation-system), and this is also the one that makes most sense in our case, since we do not have a lot of information of about each individual (we do not have many purchases from each person). 